## **FUNDMANAGERDETAILS**

This table simulates the relationship between 100 VC funds and 80 senior fund managers. Each fund is assigned exactly two managers, and each manager can oversee up to three funds (possibly with a different role in each). Manager names and experience levels are realistic and validated for integrity.


### **Column Overview**

| Column          | Example     | Notes                                                                 |
| --------------- | ----------- | --------------------------------------------------------------------- |
| FundID          | FND0024     | Unique fund identifier                                                |
| ManagerID       | MNGR017     | Unique manager code; links to manager profile                         |
| ManagerName     | Alex Garcia | Realistic U.S. name, from `randomuser.me`                             |
| Position        | Principal   | Assigned per fund: Managing Partner, Principal, or Investment Partner |
| Rank            | 1           | Indicates seniority for this fund (1 or 2)                            |
| YearsOnFund     | 6           | Number of years manager is allocated to this fund                     |
| YearsExperience | 22          | Total years of manager experience (range: 15–30)                      |


### **Assignment Logic**



1. **Manager Pool Creation**  
   - Fetch 80 unique U.S. names from the randomuser.me API.
   - Assign each a `ManagerID` (`MNGR001`–`MNGR050`).
   - Assign a random YearsExperience (15–30) to each manager

2. **Fund List**  
   - Simulate 100 unique funds: `FND0001`–`FND0100`.

3. **Assignment Rules**  
   - For each fund:
     - Select two distinct managers who:

        - Are not yet assigned to more than three funds

        - Have enough remaining experience to allocate at least one year
     - Assign a random position (`Managing Partner`, `Principal`, or `Investment Partner`) for that fund.
     - Assign a random role (`Managing Partner`, `Principal`, or `Investment Partner``) for each fund assignment
     - Randomly assign a "Rank" (1 or 2) for display order
     - Randomly allocate “YearsOnFund” (at least 1, up to available experience)

4. **Validation**
   - No fund has duplicate manager assignments.
   - No manager oversees more than three funds.
   - Each fund always has two managers.
   - Assigned years per manager never exceed total experience.

In [13]:
import requests
import pandas as pd
import random
from IPython.display import display, HTML

# Constants
POSITIONS = ["Managing Partner", "Principal", "Investment Partner"]
MIN_EXP, MAX_EXP = 15, 30
N_FUNDS = 100
FUND_IDS = [f"FND{str(i).zfill(4)}" for i in range(1, N_FUNDS+1)]
N_MANAGERS = 80  # Number of Names pulled from API for Fund Managers

# Step 1: Fetch Fund Managers
def fetch_manager_pool(n_managers):
    """Fetch manager names & assign experience. Return DataFrame."""
    r = requests.get(f"https://randomuser.me/api/?results={n_managers}&nat=us")
    r.raise_for_status()
    users = r.json()["results"]
    names = [f"{u['name']['first']} {u['name']['last']}" for u in users]
    exp = [random.randint(MIN_EXP, MAX_EXP) for _ in range(n_managers)]
    ids = [f"MNGR{str(i+1).zfill(3)}" for i in range(n_managers)]
    return pd.DataFrame({
        "ManagerID": ids,
        "ManagerName": names,
        "YearsExperience": exp
    })

df_managers = fetch_manager_pool(N_MANAGERS)

# Step 2: Track Assignments
# Each manager can be assigned to max 3 funds
assign_counts = {mid: 0 for mid in df_managers["ManagerID"]}
years_left = {mid: int(exp) for mid, exp in zip(df_managers["ManagerID"], df_managers["YearsExperience"])}

# Step 3: Assign Managers to Funds
records = []
for fund in FUND_IDS:
    # Select two available managers (assigned <3 times and still have years left)
    available = [mid for mid, cnt in assign_counts.items() if cnt < 3 and years_left[mid] > 0]
    # If not enough managers, re-use from pool (guaranteed to work with above check)
    if len(available) < 2:
        available = [mid for mid in assign_counts if assign_counts[mid] < 3 and years_left[mid] > 0]
    chosen = random.sample(available, 2)
    random.shuffle(chosen)  # Randomize order for rank
    for rank, mid in enumerate(chosen, 1):
        assign_counts[mid] += 1
        mgr = df_managers[df_managers["ManagerID"] == mid].iloc[0]
        role = random.choice(POSITIONS)
        # Each assignment: Use at least 1 year, at most what's left minus #future assignments needed
        remaining = years_left[mid]
        assignments_left = 3 - assign_counts[mid] + 1
        max_years = remaining - (assignments_left - 1)
        years_on_fund = random.randint(1, max(max_years, 1))
        years_left[mid] -= years_on_fund
        records.append({
            "FundID": fund,
            "ManagerID": mid,
            "ManagerName": mgr["ManagerName"],
            "Position": role,
            "Rank": rank,
            "YearsOnFund": years_on_fund,
            "YearsExperience": mgr["YearsExperience"]
        })

# Step 4: Present Output
df_assignments = pd.DataFrame(records)
html = df_assignments.to_html(index=False, classes="assign-table")
styled = f"""
<style>
  .assign-table th {{ text-align: center !important; }}
  .assign-table td {{ text-align: left   !important; }}
</style>
{html}
"""
display(HTML(styled))

FundID,ManagerID,ManagerName,Position,Rank,YearsOnFund,YearsExperience
FND0001,MNGR009,Terrance Pierce,Principal,1,11,26
FND0001,MNGR002,Amanda Marshall,Managing Partner,2,15,17
FND0002,MNGR073,Liam Jordan,Managing Partner,1,1,15
FND0002,MNGR072,Billy Ramirez,Investment Partner,2,15,30
FND0003,MNGR079,Lloyd Jimenez,Investment Partner,1,9,25
FND0003,MNGR036,Penny Mccoy,Principal,2,8,16
FND0004,MNGR051,Dawn Washington,Managing Partner,1,10,25
FND0004,MNGR063,Cory Chavez,Principal,2,1,15
FND0005,MNGR069,Bobby Sanchez,Investment Partner,1,18,21
FND0005,MNGR076,Margie Welch,Investment Partner,2,9,23


In [ ]:
# -- Snowflake SQL table creation for VC Fund Manager Assignments

# CREATE TABLE FUND_MANAGER_FUND_ASSIGNMENTS (
#     FUND_ID           VARCHAR(8)      NOT NULL,           -- Fund code (e.g., FND0001)
#     MANAGER_ID        VARCHAR(8)      NOT NULL,           -- Manager code (e.g., MNGR001)
#     MANAGERNAME       VARCHAR(100)    NOT NULL,           -- Full name
#     POSITION          VARCHAR(50)     NOT NULL,           -- Managing Partner / Principal / Investment Partner
#     RANK              NUMBER(1)       NOT NULL,           -- 1 or 2; indicates manager order for the fund
#     YEARSONFUND       NUMBER          NOT NULL,           -- Years allocated for this assignment
#     YEARSEXPERIENCE   NUMBER          NOT NULL,           -- Total years of experience (15–30)

#     PRIMARY KEY (FUND_ID, MANAGER_ID)                     -- Each manager-fund assignment is unique
# );

---